# **Fintech SaaS Churn Intelligence**
## **Notebook 02 - Feature Engineering**

This notebook begins the predictive modeling workflow using the processed datasets from Notebook 01.  
Our focus here is to build high-signal behavioral features and prepare a clean modeling dataset.

### **Objectives**
1. Load curated user-level and synthetic event-level datasets  
2. Engineer behavioral, recency, and engagement features  
3. Build a unified modeling dataset for churn prediction  
4. Perform sanity checks before modeling  

By the end of this section, we will have a robust feature table ready for model training.


## IMPORT LIBRARIES

In [17]:
# Standard imports and environment setup
import os
import pandas as pd            # dataframes
import numpy as np             # numerical ops
from datetime import datetime, timedelta
import random

# For fast SQL-like queries if needed later
import duckdb

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, precision_recall_curve


# Basic utilities
import warnings
warnings.filterwarnings("ignore")

# Notebook display niceties
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 140)
sns.set(style='whitegrid')

print("Imports ready. Pandas version:", pd.__version__)


Imports ready. Pandas version: 2.3.3


## 1. Create synthetic event aggregates 
We ensure all IDs share a consistent `string` dtype.


In [20]:
# Rebuild agg_telco from remapped events or from original events (with remapping fallback)

processed_dir = "../data/processed"
synthetic_dir = "../data/synthetic"
os.makedirs(processed_dir, exist_ok=True)

# candidate remapped file (balanced preferred)
balanced_path = os.path.join(synthetic_dir, "events_telco_mapped_balanced.csv")
remapped_path = os.path.join(synthetic_dir, "events_telco_mapped.csv")
original_events_path = os.path.join(synthetic_dir, "events.csv")

# decide which events file to use
if os.path.exists(balanced_path):
    events_src = balanced_path
    print("Using balanced remapped events:", balanced_path)
elif os.path.exists(remapped_path):
    events_src = remapped_path
    print("Using remapped events:", remapped_path)
elif os.path.exists(original_events_path):
    events_src = original_events_path
    print("Using original synthetic events file (events.csv); will remap to telco IDs below:", original_events_path)
else:
    raise FileNotFoundError("No events file found. Place a synthetic events CSV under data/synthetic/ named events.csv or events_telco_mapped_balanced.csv")

# load events (do not assume signup_date exists)
events = pd.read_csv(events_src, low_memory=False, parse_dates=['timestamp'], infer_datetime_format=True)
events['user_id'] = events['user_id'].astype(str)

# If using original events.csv (no remapping), attempt to remap now using users list.
if events_src.endswith("events.csv"):
    # load users to remap (if available)
    users_path = os.path.join(processed_dir, "users_normalized.csv")
    if not os.path.exists(users_path):
        # try raw users
        raw_users_path = "../data/raw/telco_churn.csv"
        if os.path.exists(raw_users_path):
            users = pd.read_csv(raw_users_path, low_memory=False)
            # ensure a user_id column exists; try common names
            if 'user_id' not in users.columns:
                if 'customerID' in users.columns:
                    users['user_id'] = users['customerID'].astype(str)
                else:
                    # create synthetic user ids from index (last-resort)
                    users['user_id'] = users.index.astype(str)
            # save normalized users file for reproducibility
            users[['user_id']].to_csv(users_path, index=False)
            print("Created minimal users_normalized.csv from raw telco file.")
        else:
            raise FileNotFoundError("Cannot remap events because users table not found. Place users_normalized.csv under data/processed/ or telco raw under data/raw/.")

    # load canonical users and remap events balancedly (guarantee >=1 per user)
    users = pd.read_csv(users_path, low_memory=False)
    telco_ids = users['user_id'].astype(str).unique().tolist()
    n_telco = len(telco_ids)
    n_events = len(events)
    print(f"Remapping {n_events} events to {n_telco} telco users (guarantee >=1 per user).")

    # simple balanced remap: sample one event per user then distribute remainder with weights
    rng = np.random.default_rng(42)

    if n_events < n_telco:
        raise ValueError("Not enough events to guarantee one-per-user. Generate more synthetic events first.")

    # 1) pick sample indices for guaranteed assignment
    guaranteed_idx = events.sample(n=n_telco, random_state=42).index
    events_remap = events.copy()
    events_remap.loc[guaranteed_idx, 'user_id'] = telco_ids  # assign 1-to-1 deterministically via order

    # 2) remaining events: weighted assignment (power-law like)
    remaining_mask = ~events.index.isin(guaranteed_idx)
    remaining_indices = events.index[remaining_mask]
    top = int(0.10 * n_telco)
    mid = int(0.40 * n_telco)
    rest = n_telco - (top + mid)
    weights = np.concatenate([np.full(top, 5.0), np.full(mid, 2.0), np.full(rest, 1.0)])
    weights = weights / weights.sum()
    # sample telco ids for remaining events
    sampled = rng.choice(telco_ids, size=len(remaining_indices), replace=True, p=weights)
    events_remap.loc[remaining_indices, 'user_id'] = sampled

    events = events_remap
    # persist remapped events for reproducibility
    out_remapped = os.path.join(synthetic_dir, "events_telco_mapped_balanced.csv")
    events.to_csv(out_remapped, index=False)
    print("Saved remapped events to:", out_remapped)

# At this point 'events' is the remapped event log (user_id = telco ids). Build aggregates:
events['timestamp'] = pd.to_datetime(events['timestamp'], errors='coerce')
events['active_day'] = events['timestamp'].dt.date

# Compute aggregates
events_count = events.groupby('user_id').size().rename('events_count').reset_index()
unique_active_days = events.groupby('user_id')['active_day'].nunique().rename('unique_active_days').reset_index()
first_event = events.groupby('user_id')['timestamp'].min().rename('first_event_ts').reset_index()
last_event  = events.groupby('user_id')['timestamp'].max().rename('last_event_ts').reset_index()
support_cnt = events.loc[events['event_type']=='support_contact'].groupby('user_id').size().rename('support_contacts_count').reset_index()
error_cnt   = events.loc[events['event_type']=='error_event'].groupby('user_id').size().rename('error_event_count').reset_index()

reference_date = pd.to_datetime("2023-12-31")
events_last30 = events[events['timestamp'] >= (reference_date - pd.Timedelta(days=30))]
events_30d = events_last30.groupby('user_id').size().rename('events_last_30d').reset_index()

# Merge into tidy agg_telco
agg_telco = events_count.merge(unique_active_days, on='user_id', how='left') \
                        .merge(first_event, on='user_id', how='left') \
                        .merge(last_event, on='user_id', how='left') \
                        .merge(support_cnt, on='user_id', how='left') \
                        .merge(error_cnt, on='user_id', how='left') \
                        .merge(events_30d, on='user_id', how='left')

# fill NaNs for counts with 0 (no activity)
for c in ['events_count','unique_active_days','support_contacts_count','error_event_count','events_last_30d']:
    if c in agg_telco.columns:
        agg_telco[c] = agg_telco[c].fillna(0).astype(int)

# Save agg_telco
agg_path = os.path.join(processed_dir, "agg_telco.csv")
agg_telco.to_csv(agg_path, index=False)
print("agg_telco written to", agg_path)
display(agg_telco.head())


Using balanced remapped events: ../data/synthetic/events_telco_mapped_balanced.csv
agg_telco written to ../data/processed/agg_telco.csv


,user_id,events_count,unique_active_days,first_event_ts,last_event_ts,support_contacts_count,error_event_count,events_last_30d
0,0002-ORFBO,56,51,2023-02-10 05:39:48,2024-10-28 09:34:14,2,3,16
1,0003-MKNFE,65,63,2023-02-10 19:34:47,2025-01-13 18:46:13,0,5,21
2,0004-TLHLJ,27,26,2023-02-17 17:23:27,2024-12-09 19:02:28,1,1,7
3,0011-IGKFF,31,29,2023-03-13 15:42:47,2024-08-04 02:21:28,1,2,7
4,0013-EXCHZ,38,38,2023-01-06 20:47:02,2024-05-07 02:04:11,0,0,10


## 2. Load User-Level and Event-Level Data  
We begin by loading:

- `users` → Telco customer dataset with normalized churn fields (from Notebook 01)  
- `agg_telco` → Synthetic event aggregates computed in Notebook 01  
- `events` → Raw synthetic events (optional, useful for inspection)

We ensure all IDs share a consistent `string` dtype.


In [21]:
# Step: Load users_normalized and agg_telco with safety checks
import pandas as pd, os

users_path = "../data/processed/users_normalized.csv"
agg_path = "../data/processed/agg_telco.csv"

if not os.path.exists(users_path):
    # try alternative names: users_features_telco_mapped.csv or users_features.csv
    alt = "../data/processed/users_features_telco_mapped.csv"
    if os.path.exists(alt):
        users_path = alt
        print("Using alternative users file:", alt)
    else:
        raise FileNotFoundError("users_normalized.csv not found under data/processed/. Please run Notebook 01 or provide users table.")

users = pd.read_csv(users_path, parse_dates=['start_date','churn_date'], low_memory=False)
agg_telco = pd.read_csv(agg_path, low_memory=False)

# normalize id types
users['user_id'] = users['user_id'].astype(str)
agg_telco['user_id'] = agg_telco['user_id'].astype(str)

print("Loaded users:", users.shape)
print("Loaded agg_telco:", agg_telco.shape)
display(agg_telco.head())


Loaded users: (7043, 28)
Loaded agg_telco: (7043, 8)


,user_id,events_count,unique_active_days,first_event_ts,last_event_ts,support_contacts_count,error_event_count,events_last_30d
0,0002-ORFBO,56,51,2023-02-10 05:39:48,2024-10-28 09:34:14,2,3,16
1,0003-MKNFE,65,63,2023-02-10 19:34:47,2025-01-13 18:46:13,0,5,21
2,0004-TLHLJ,27,26,2023-02-17 17:23:27,2024-12-09 19:02:28,1,1,7
3,0011-IGKFF,31,29,2023-03-13 15:42:47,2024-08-04 02:21:28,1,2,7
4,0013-EXCHZ,38,38,2023-01-06 20:47:02,2024-05-07 02:04:11,0,0,10


## 3. Clean Duplicate Columns  
During earlier processing, exploratory merges can introduce duplicate column names, which cause:

- SHAP failures  
- Pandas merge inconsistencies  
- Feature collisions in ML models  

We apply a strict deduplication rule to ensure every column name is unique.


In [22]:
def drop_duplicate_columns(df):
    """Ensure each column name appears once."""
    return df.loc[:, ~df.columns.duplicated()]

users = drop_duplicate_columns(users)
agg_telco = drop_duplicate_columns(agg_telco)

print("Users columns:", len(users.columns))
print("Agg_telco columns:", len(agg_telco.columns))


Users columns: 28
Agg_telco columns: 8


## 4. Merge Synthetic Behavioral Features Into Telco Users  
We perform a **single clean merge**:

- Left table: Telco users  
- Right table: aggregated synthetic behavior  
- Suffix `_agg` added only where a name collision occurs  
- All duplicate columns dropped after merge  

This gives us a unified modeling dataset with both demographic and behavioral signals.


In [23]:
users_merged = users.merge(
    agg_telco,
    on="user_id",
    how="left",
    suffixes=('', '_agg')
)

# Drop any remaining duplicates
users_merged = drop_duplicate_columns(users_merged)

print("Merged dataset shape:", users_merged.shape)


Merged dataset shape: (7043, 35)


## 4. Consolidate / Overwrite Aggregated Feature Columns  
If a feature appears both in Telco and in the synthetic aggregates, we:

1. Prefer the event-derived (`_agg`) version  
2. Overwrite the base version  
3. Fill missing counts with zero  
4. Leave timestamps as NaT until processed in the next step  


In [24]:
# ===================================================================
# Standardization and type-correction of count-based behavioral features
# ===================================================================

# Define the list of count features that represent user activity, support usage, and errors
# These features are critical behavioral signals for churn prediction and engagement analysis
count_features = [
    'events_count',          # Total number of events performed by the user
    'unique_active_days',    # Number of distinct calendar days the user was active
    'support_contacts_count',# Number of times the user contacted customer support
    'error_event_count',     # Number of error or failure events recorded
    'events_last_30d',       # Number of events in the most recent 30-day window
]

# Iterate over each count feature to ensure data quality and consistency
for col in count_features:
    
    # Check for legacy/aggregated column names (e.g., from previous pipeline versions)
    # Some features may have been computed under a different name (e.g., 'events_count_agg')
    agg_col = col + "_agg"
    if agg_col in users_merged.columns:
        # Prefer the explicitly aggregated version if it exists
        users_merged[col] = users_merged[agg_col]
    
    # Ensure all count features are integer-typed and handle missing values
    # - Missing activity = 0 (user was inactive or never triggered the event)
    # - Integer type is required for interpretability and modeling stability
    users_merged[col] = (
        users_merged[col]
        .fillna(0)          # Replace NaN (no activity) with zero
        .astype(int)         # Convert to integer (e.g., 3.0 → 3)
    )

# Post-processing verification 
print("Count features after standardization:")
print(users_merged[count_features].dtypes)
print(users_merged[count_features].describe())

Count features after standardization:
events_count              int64
unique_active_days        int64
support_contacts_count    int64
error_event_count         int64
events_last_30d           int64
dtype: object
       events_count  unique_active_days  support_contacts_count  error_event_count  events_last_30d
count   7043.000000         7043.000000             7043.000000        7043.000000      7043.000000
mean      53.825075           49.964930                1.077240           2.133466        15.761607
std       34.984385           29.571455                1.259555           1.990768        10.819822
min       13.000000           13.000000                0.000000           0.000000         1.000000
25%       30.000000           29.000000                0.000000           1.000000         9.000000
50%       44.000000           42.000000                1.000000           2.000000        13.000000
75%       62.000000           58.000000                2.000000           3.000000      

## 5. Construct Time-Based Behavioral Features  
These features are crucial churn predictors:

- **Time to First Event:** Measures onboarding friction  
- **Days Since Last Event:** Recency → strongest churn signal  

We also introduce a sentinel value `999` to indicate *no activity recorded*.


### 4. Quick validation of engineered features

Run simple diagnostics and plots to confirm the behavioral features now have variance (not all zeros).  
We check:
- unique counts per feature (should be >1 for most features)
- min/max values
- small histograms for sanity of distributions

If these checks pass, proceed to modeling.


In [25]:
# Set analysis reference date (last observed date)
reference_date = pd.to_datetime("2023-12-31")

# Reconstruct first/last event timestamps from aggregated columns if available
for ts in ['first_event_ts', 'last_event_ts']:
    ts_agg = ts + "_agg"
    if ts_agg in users_merged.columns:
        # Use pre-aggregated timestamp if exists; coerce errors to NaT
        users_merged[ts] = pd.to_datetime(users_merged[ts_agg], errors='coerce')
    else:
        # No event data → mark as missing
        users_merged[ts] = pd.NaT

# Compute key behavioral timing features (in days)
users_merged['time_to_first_event_days'] = (
    (users_merged['first_event_ts'] - users_merged['start_date']).dt.days
    .fillna(999)  # 999 = never had first event (high penalty value)
)

users_merged['days_since_last_event'] = (
    (reference_date - users_merged['last_event_ts']).dt.days
    .fillna(999)  # 999 = never active or no last event
)

# Confirmation: display summary of timing features
print("Timing features computed:")
print(users_merged[['time_to_first_event_days', 'days_since_last_event']].describe().round(1))
print(f"\nUsers with no first event: {(users_merged['time_to_first_event_days'] == 999).sum():,}")
print(f"Users with no recent activity: {(users_merged['days_since_last_event'] == 999).sum():,}")

Timing features computed:
       time_to_first_event_days  days_since_last_event
count                    7043.0                 7043.0
mean                      999.0                  999.0
std                         0.0                    0.0
min                       999.0                  999.0
25%                       999.0                  999.0
50%                       999.0                  999.0
75%                       999.0                  999.0
max                       999.0                  999.0

Users with no first event: 7,043
Users with no recent activity: 7,043


## 6. Final Cleanup & Verification  
We drop helper columns ending in `_agg` and inspect a random sample to verify:

- Non-zero behavioral features  
- Correct time delta calculations  
- Aligned and clean user IDs  
- No duplicate columns  

This dataset is now fully ready for modeling.


In [34]:
# Drop helper columns
agg_cols = [c for c in users_merged.columns if c.endswith('_agg')]
users_final = users_merged.drop(columns=agg_cols)

print("Final modeling dataset shape:", users_final.shape)

# Preview a few rows
display(users_final.sample(10, random_state=42))

# Replace canonical users dataset
users = users_final.copy()

# Save the final processed dataset for use in modeling notebook
users.to_csv('../data/processed/users_final.csv', index=False)
print(f"Saved processed dataset with {len(users)} rows to ../data/processed/users_final.csv")


Final modeling dataset shape: (7043, 37)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,user_id,start_date,churned,churn_date,end_date,tenure_days,tenure_months_precise,events_count,unique_active_days,first_event_ts,last_event_ts,support_contacts_count,error_event_count,events_last_30d,time_to_first_event_days,days_since_last_event
185,1024-GUALD,Female,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,24.80,24.80,Yes,1024-GUALD,2023-11-30,True,2023-12-30,2023-12-30,30,0.99,136,118,NaT,NaT,0,8,37,999.0,999.0
2715,0484-JPBRU,Male,0,No,No,41,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Bank transfer (automatic),25.25,996.45,No,0484-JPBRU,2020-07-31,False,NaT,2023-12-31,1248,41.00,67,65,NaT,NaT,2,0,22,999.0,999.0
3825,3620-EHIMZ,Female,0,Yes,Yes,52,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.35,1031.70,No,3620-EHIMZ,2019-08-31,False,NaT,2023-12-31,1583,52.00,36,35,NaT,NaT,1,0,11,999.0,999.0
1807,6910-HADCM,Female,0,No,No,1,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Electronic check,76.35,76.35,Yes,6910-HADCM,2023-11-30,True,2023-12-30,2023-12-30,30,0.99,63,57,NaT,NaT,1,2,16,999.0,999.0
132,8587-XYZSF,Male,0,No,No,67,Yes,No,DSL,No,No,No,Yes,No,No,Two year,No,Bank transfer (automatic),50.55,3260.10,No,8587-XYZSF,2018-05-31,False,NaT,2023-12-31,2040,67.02,155,137,NaT,NaT,4,4,49,999.0,999.0
1263,6818-WOBHJ,Female,1,Yes,No,68,Yes,Yes,Fiber optic,No,Yes,No,No,No,Yes,Month-to-month,Yes,Bank transfer (automatic),89.60,6127.60,Yes,6818-WOBHJ,2018-04-30,True,2023-12-30,2023-12-30,2070,68.00,52,52,NaT,NaT,1,2,19,999.0,999.0
3732,3082-YVEKW,Female,0,Yes,Yes,23,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),77.15,1759.40,No,3082-YVEKW,2022-01-31,False,NaT,2023-12-31,699,22.96,31,29,NaT,NaT,0,1,5,999.0,999.0
1672,4737-AQCPU,Male,0,Yes,Yes,72,Yes,Yes,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Credit card (automatic),72.10,5016.65,No,4737-AQCPU,2017-12-31,False,NaT,2023-12-31,2191,71.98,53,51,NaT,NaT,0,2,13,999.0,999.0
811,4853-RULSV,Male,0,No,No,70,Yes,Yes,Fiber optic,Yes,No,No,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),104.00,7250.15,Yes,4853-RULSV,2018-02-28,True,2023-12-28,2023-12-28,2129,69.94,50,49,NaT,NaT,1,2,14,999.0,999.0
2526,5766-ZJYBB,Male,0,No,No,1,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.40,19.40,Yes,5766-ZJYBB,2023-11-30,True,2023-12-30,2023-12-30,30,0.99,72,65,NaT,NaT,2,1,26,999.0,999.0


Saved processed dataset with 7043 rows to ../data/processed/users_final.csv
